In [17]:
# Download the data
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

# Import Data

In [1]:
import numpy as np
import pandas as pd

In [2]:
y_jan = pd.read_parquet('yellow_tripdata_2023-01.parquet')
y_feb = pd.read_parquet('yellow_tripdata_2023-02.parquet')

Q1. Downloading the data

In [3]:
print(y_jan.shape)
y_jan.head()

(3066766, 19)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


Q2. Computing duration

In [4]:
# Compute duration in minutes for January trips
y_jan['duration'] = (y_jan['tpep_dropoff_datetime'] - y_jan['tpep_pickup_datetime']).dt.total_seconds() / 60

# Calculate standard deviation of duration
std_duration_jan = y_jan['duration'].std()
print(f"Standard deviation of trip duration in January: {std_duration_jan:.2f} minutes")

Standard deviation of trip duration in January: 42.59 minutes


Q3. Dropping outliers

In [5]:
# Filter y_jan to keep only durations between 1 and 60 minutes (inclusive)
mask = (y_jan['duration'] >= 1) & (y_jan['duration'] <= 60)
filtered_y_jan = y_jan[mask]

# Calculate the fraction of records left
fraction_left = len(filtered_y_jan) / len(y_jan)
print(f"Fraction of records left after dropping outliers: {fraction_left:.3f}")

Fraction of records left after dropping outliers: 0.981


Q4. One-hot encoding

In [6]:
from sklearn.feature_extraction import DictVectorizer

# Select only pickup and dropoff location IDs, cast to string
features = filtered_y_jan[['PULocationID', 'DOLocationID']].astype(str)

# Convert to list of dictionaries
dicts = features.to_dict(orient='records')

# Fit DictVectorizer
dv = DictVectorizer()
X = dv.fit_transform(dicts)

# Print the dimensionality (number of columns)
print(f"Feature matrix shape: {X.shape}")
print(f"Number of columns: {X.shape[1]}")

Feature matrix shape: (3009173, 515)
Number of columns: 515


Q5. Training a model

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Prepare target variable
y_train = filtered_y_jan['duration'].values

# Train linear regression model
lr = LinearRegression()
lr.fit(X, y_train)

# Predict on training data
y_pred = lr.predict(X)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print(f"RMSE on train: {rmse:.2f}")

RMSE on train: 7.65


Q6. Evaluating the model

In [8]:
# Compute duration in minutes for February trips
y_feb['duration'] = (y_feb['tpep_dropoff_datetime'] - y_feb['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter y_feb to keep only durations between 1 and 60 minutes (inclusive)
mask_feb = (y_feb['duration'] >= 1) & (y_feb['duration'] <= 60)
filtered_y_feb = y_feb[mask_feb]

In [9]:
# Prepare features for validation set
features_feb = filtered_y_feb[['PULocationID', 'DOLocationID']].astype(str)
dicts_feb = features_feb.to_dict(orient='records')
X_val = dv.transform(dicts_feb)

# Prepare target variable for validation set
y_val = filtered_y_feb['duration'].values

In [10]:
# Predict and calculate RMSE
y_pred_val = lr.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"RMSE on validation: {rmse_val:.2f}")

RMSE on validation: 7.81
